# Cargando librerias

In [76]:
import boto3
from boto3.dynamodb.conditions import Attr, Key
import pandas as pd
import numpy as np
import json

In [29]:
dynamodb= boto3.resource('dynamodb')
ordenes = dynamodb.Table('Ordenes')
productos = dynamodb.Table('productos')

## Obtener los IDs de todos los productos

In [31]:
response2 = productos.scan(
    ProjectionExpression = 'ProductID'
)
item2 = response2['Items']
productIDs = list()
for item in item2:
    productIDs.append(item['ProductID'])
print(productIDs)

['BF1772', 'BF2152', 'BF1681', 'BF2370', 'BF9289', 'BF8238', 'BF0060', 'BF9878', 'BF0809', 'BF7814', 'BF8063', 'BF6885', 'BF5238', 'BF2887', 'BF4831', 'BF9724', 'BF6693', 'BF1414', 'BF0103', 'BF6167', 'BF6975', 'BF6604', 'BF3942', 'BF9708']


## Creando la tabla

In [88]:
R = pd.DataFrame(columns = ['ClientID'] + productIDs)
R.set_index('ClientID')
R

,ClientID,BF1772,BF2152,BF1681,BF2370,BF9289,BF8238,BF0060,BF9878,BF0809,...,BF4831,BF9724,BF6693,BF1414,BF0103,BF6167,BF6975,BF6604,BF3942,BF9708


## Agregando datos a la tabla
### Datos de la primera pagina

In [89]:
response = ordenes.scan(
    ProjectionExpression = 'ClientID, ProductID, Rating'
)

orders = response['Items']
last = response['LastEvaluatedKey']

for order in orders:
    product = order['ProductID']
    Client = order['ClientID']
    rating = float(order['Rating'])
    R.loc[len(R)] = {'ClientID': Client, product: rating}

R

,ClientID,BF1772,BF2152,BF1681,BF2370,BF9289,BF8238,BF0060,BF9878,BF0809,...,BF4831,BF9724,BF6693,BF1414,BF0103,BF6167,BF6975,BF6604,BF3942,BF9708
0,601355057,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,190499590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,679633181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,153154348,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,220575020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,442417730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3450,112137859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3451,766437236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
3452,586912408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN


### Datos de siguientes paginas

In [90]:
paginating = True
while paginating == True:
        response = ordenes.scan(
            ExclusiveStartKey = last,
            ProjectionExpression = 'ClientID, ProductID, Rating'
        )

        orders = response['Items']

        for order in orders:
            product = order['ProductID']
            Client = order['ClientID']
            rating = float(order['Rating'])
            R.loc[len(R)] = {'ClientID': Client, product: rating}
        
        try: 
            last = response['LastEvaluatedKey']
        except: 
             paginating = False


R

,ClientID,BF1772,BF2152,BF1681,BF2370,BF9289,BF8238,BF0060,BF9878,BF0809,...,BF4831,BF9724,BF6693,BF1414,BF0103,BF6167,BF6975,BF6604,BF3942,BF9708
0,601355057,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,190499590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,679633181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,153154348,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,220575020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7389,190491552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7390,228569299,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7391,472421009,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7392,293357104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Colapsamos las filas para que cada cliente sea una fila.

In [95]:
coll_R = R.groupby('ClientID').max()
coll_R

,BF1772,BF2152,BF1681,BF2370,BF9289,BF8238,BF0060,BF9878,BF0809,BF7814,...,BF4831,BF9724,BF6693,BF1414,BF0103,BF6167,BF6975,BF6604,BF3942,BF9708
ClientID,,,,,,,,,,,,,,,,,,,,,
100071193,NaN,NaN,NaN,5.0,NaN,NaN,NaN,4.0,NaN,4.0,...,NaN,4.0,4.0,2.0,4.0,NaN,NaN,3.0,1.0,2.0
102228703,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,...,NaN,4.0,5.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN
102320254,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN
103076257,1.0,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,...,NaN,4.0,3.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN
103692255,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897199887,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,2.0,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
897330680,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,...,NaN,5.0,NaN,5.0,NaN,NaN,5.0,5.0,NaN,5.0
898088860,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,1.0,NaN,4.0,NaN,NaN,NaN


Rellenamos campos faltantes

In [97]:
coll_R.replace(to_replace=np.nan, value=0, inplace=True)
coll_R

,BF1772,BF2152,BF1681,BF2370,BF9289,BF8238,BF0060,BF9878,BF0809,BF7814,...,BF4831,BF9724,BF6693,BF1414,BF0103,BF6167,BF6975,BF6604,BF3942,BF9708
ClientID,,,,,,,,,,,,,,,,,,,,,
100071193,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0,4.0,...,0.0,4.0,4.0,2.0,4.0,0.0,0.0,3.0,1.0,2.0
102228703,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
102320254,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,5.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0
103076257,1.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,...,0.0,4.0,3.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
103692255,0.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897199887,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,2.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
897330680,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,...,0.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0,0.0,5.0
898088860,0.0,0.0,0.0,4.0,5.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0


Y guardamos en csv

In [99]:
coll_R.to_csv("../../Data Sets/matrix_ratings.csv")

---

NOT REAL

In [ ]:
histComp_df = coll_R.copy()
for prod in productIDs:
    histComp_df[prod] = histComp_df[prod].apply(lambda x: True if x== 0 else False)

histComp_df.to_csv("../../Data Sets/matrix_histCompras.csv")
histComp_df

,BF1772,BF2152,BF1681,BF2370,BF9289,BF8238,BF0060,BF9878,BF0809,BF7814,...,BF4831,BF9724,BF6693,BF1414,BF0103,BF6167,BF6975,BF6604,BF3942,BF9708
ClientID,,,,,,,,,,,,,,,,,,,,,
100071193,True,True,True,False,True,True,True,False,True,False,...,True,False,False,False,False,True,True,False,False,False
102228703,True,True,True,True,False,True,True,False,True,True,...,True,False,False,True,True,True,True,False,True,True
102320254,True,False,False,True,True,True,True,False,True,True,...,True,True,False,False,True,True,True,False,True,True
103076257,False,False,True,True,False,True,True,True,True,False,...,True,False,False,False,True,False,True,True,True,True
103692255,True,True,False,True,True,True,False,True,True,True,...,True,False,False,True,True,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897199887,True,True,True,False,False,True,True,True,True,False,...,True,False,True,True,True,True,True,False,True,True
897330680,True,True,False,True,True,False,True,True,False,True,...,True,False,True,False,True,True,False,False,True,False
898088860,True,True,True,False,False,True,False,True,True,True,...,True,True,False,True,False,True,False,True,True,True
